In [323]:
# Import necessary libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime

def get_match_data(start_date,time_now_formatted,devengine):


    # Get historical match data on hard surface between start date and yesterday
    elo_hard = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )

    # Get historical match data on clay surface between start date and yesterday
    elo_clay = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    # Get historical match data on clay surface between start date and yesterday
    elo_grass = pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date > '{start_date}' and Date not like '{time_now_formatted}'",
        con=devengine,
    )
    #Get today's matches on hard surface that haven't yet been resulted
    elo_data_hard =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,)

    #Get today's matches on clay surface that haven't yet been resulted
    elo_data_clay =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,)
    
    #Get today's matches on clay surface that haven't yet been resulted
    elo_data_grass =pd.read_sql_query(
        f"Select DISTINCT * From Elo_AllMatches_grass where Date like '{time_now_formatted}' --and resulted like 'False'",
        con=devengine,)
    return elo_hard, elo_clay, elo_data_hard, elo_data_clay,elo_grass,elo_data_grass

# Connect to SQLite database using SQLAlchemy's create_engine
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
# Get current date and time
time_now = datetime.datetime.now()

# Format current date as string in YYYY-MM-DD format
time_now_formatted = time_now.strftime("%Y-%m-%d")

# Get the start date two years ago from today
today =time_now
two_years_ago = (today - datetime.timedelta(days=365*2)).strftime("%Y-%m-%d")

elo_hard, elo_clay, elo_data_hard, elo_data_clay,elo_grass,elo_data_grass = get_match_data(two_years_ago,time_now_formatted,devengine)

def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0

def get_filtered_data(elo_data, elo):
    result_df = pd.DataFrame()
    for _, row in elo_data.sort_values(by="Time").iterrows():
        low_limit = 50
        high_limit = 50

        fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            fav_percent, games = get_player_record(row.Fav, row.Dog_Rank, elo_hard, low_limit, high_limit, True)

        low_limit = 50
        high_limit = 50
        dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)
        count = 0
        while games2 < 10 and count < 200:
            count = count + 1
            low_limit = low_limit + 10
            high_limit = high_limit + 10
            dog_percent, games2 = get_player_record(row.Dog, row.Fav_Rank, elo_hard, low_limit, high_limit, True)

        if games > 4 and games2 > 4:
            temp_df = pd.DataFrame(
                {
                    "Time": [row.Time],
                    "Fav_Odds": [row.Fav_Odds],
                    "Dog_Odds": [row.Dog_Odds],
                    "Fav": [row.Fav],
                    "Elo_Fav": [row.Elo_Fav],
                    "Fav_Record": ["{:.0%}".format(fav_percent)],
                    "Fav_Games": [games],
                    "Dog": [row.Dog],
                    "Dog_Odds": [row.Dog_Odds],
                    "Dog_Record": ["{:.0%}".format(dog_percent)],
                    "Dog_Games": [games2],
                    "fav_percent": [fav_percent],
                    "dog_percent": [dog_percent],
                    "Sex": [row.Sex],
                    "Resulted":[row.Resulted],
                    "fav_rank":[row.Fav_Rank],
                    "dog_rank":[row.Dog_Rank],
                    "Elo_Fav_Elo":[row.Elo_Fav_Elo],
                    "Elo_Dog_Elo":[row.Elo_Dog_Elo],                    
                }
            )
            result_df = pd.concat([result_df, temp_df])
    return result_df

results_hard=get_filtered_data(elo_data_hard,elo_hard)
results_clay=get_filtered_data(elo_data_clay,elo_clay)
results_grass=get_filtered_data(elo_data_grass,elo_grass)
def process_serving_data(result_df):
    try:
        # Try to read the 'servers_today.xlsx' file
        serving = pd.read_csv('mensserving.csv')

        # Try to read the 'servers_today_womens.xlsx' file
        serving_womens = pd.read_csv('womensserving.csv')

        # If 'serving_womens' dataframe is not empty, concatenate with 'serving' dataframe
        if serving_womens.empty == False:
            serving = pd.concat([serving, serving_womens])
        else:
            serving = serving

        # Drop the 'Time' column from the 'serving' dataframe
        #serving = serving.drop(columns='Time')
    except FileNotFoundError as e:
        # If either of the excel files is not found, print an error message and set serving to None
        print("The required excel file could not be found.")
        print("Error:", e)
        serving = None

    if serving is not None:
        try:
            # Try to merge the 'result_df' and 'serving' dataframes on the 'Fav' and 'Dog' columns
            result = pd.merge(result_df, serving, how='left', left_on=['Fav'], right_on=['Name'])
            result = pd.merge(result,serving, how='left', left_on=['Dog'], right_on=['Name'])
            result.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,)
            result.drop(columns=['Name_x','Name_y'],inplace=True)
    

            # Set 'final_hard' to the 'result' dataframe
            result_serving = result
        except Exception as e:
            # If an error occurs during merging, print an error message and set both 'result' and 'final_hard' to None
            print("Error occured while merging the dataframes.")
            print("Error:", e)
            result = None
            result_serving = None
    else:
        result = None
        result_serving = None
        
    return result, result_serving


_,serving_hard=process_serving_data(results_hard)
_,serving_clay=process_serving_data(results_clay)
_,serving_grass=process_serving_data(results_grass)

elo_hardxx = pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2022-01-01' and Date not like '{time_now_formatted}' --and Fav_odds>1.2 and Fav_odds<1.4 ",
    con=devengine,)

data_concat = pd.DataFrame(columns=['Date', 'Player', 'Odds', 'Win/Loss'])
for dataset_type in [('Winner', 'Win'), ('Loser', 'Loss')]:
    df = elo_hard[['Date', dataset_type[0], f"{dataset_type[0]}_Odds"]].copy()
    df['Player'] = df[dataset_type[0]]
    df['Odds'] = df[f"{dataset_type[0]}_Odds"]
    df['Win/Loss'] = dataset_type[1]
    df.drop(columns=[dataset_type[0], f"{dataset_type[0]}_Odds"], inplace=True)
    data_concat = pd.concat([data_concat, df])
data_concat = data_concat.sort_index()
data_concat['Odds'] = data_concat.Odds.astype(float)

def analyse_matchups(result_df,data_concat):
    for _,matchup in result_df.iterrows():
        player1=matchup.Fav
        player2=matchup.Dog
        player1_odds=float(matchup.Fav_Odds)
        player1_odds_hi=player1_odds+0.15
        player1_odds_lo=player1_odds-0.15
        player2_odds=float(matchup.Dog_Odds)
        player2_odds_hi=player2_odds+0.15
        player2_odds_lo=player2_odds-0.15
        player1=data_concat[data_concat['Player']==player1].copy()
        player2=data_concat[data_concat['Player']==player2].copy()
        player2=player2[(player2['Odds']>player2_odds_lo)&(player2['Odds']<player2_odds_hi)]
        if len(player2)>0:
            winperc2=len(player2[player2['Win/Loss']=='Win'])/len(player2)
        else:
            winperc2=0
        player1=player1[(player1['Odds']>player1_odds_lo)&(player1['Odds']<player1_odds_hi)]
        if len(player1)>0:
            winperc1=len(player1[player1['Win/Loss']=='Win'])/len(player1)
        else:
            winperc1=0
        if len(player1)>5 and len(player2)>3:
            print(matchup.Time,f"{matchup.Fav} ({round(player1_odds_lo,2)}-->{round(player1_odds_hi,2)})",f"{matchup.Dog} ({round(player2_odds_lo,2)}-->{round(player2_odds_hi,2)})")
            print(len(player1),winperc1,len(player2),winperc2)


#analyse_matchups(serving_clay,data_concat)
final_hard=serving_hard
final_clay=serving_clay
final_grass=serving_grass

In [324]:
def last_five(df,pastmatches):
    for index, row in df.iterrows():
        fav=row.Fav
        dog=row.Dog
        last_five_matches_fav=pastmatches[(pastmatches['Winner']==fav)|(pastmatches['Loser']==fav)].tail(5)
        if len(last_five_matches_fav)>0:  
            fav_last_five_win_perc=len(last_five_matches_fav[last_five_matches_fav['Winner']==fav])/len(last_five_matches_fav)
        else:
            fav_last_five_win_perc=0
        last_five_matches_dog=pastmatches[(pastmatches['Winner']==dog)|(pastmatches['Loser']==dog)].tail(5)
        if len(last_five_matches_dog)>0:    
            dog_last_five_win_perc=len(last_five_matches_dog[last_five_matches_dog['Winner']==dog])/len(last_five_matches_dog) 
        else:
            dog_last_five_win_perc=0
        df.at[index, 'fav_last_five_win_perc'] = fav_last_five_win_perc
        df.at[index, 'dog_last_five_win_perc'] = dog_last_five_win_perc
    return df

if final_hard is not None:
    final_hard=last_five(final_hard,elo_hard)
    final_hard['Fav_Odds']=final_hard['Fav_Odds'].astype(float)
    final_hard['Dog_Odds']=final_hard['Dog_Odds'].astype(float)    

In [325]:
if final_clay is not None:
    final_clay=last_five(final_clay,elo_clay)
    final_clay['Dog_Odds']=final_clay['Dog_Odds'].astype(float)
    final_clay['Fav_Odds']=final_clay['Fav_Odds'].astype(float)

In [326]:
if final_grass is not None:
    final_grass=last_five(final_grass,elo_grass)
    final_grass['Fav_Odds']=final_grass['Fav_Odds'].astype(float)
    final_grass['Dog_Odds']=final_grass['Dog_Odds'].astype(float)    

In [327]:
for _,i in elo_data_hard.iterrows():
    check1=elo_hard[((elo_hard['Winner']==i.Winner)&(elo_hard['Loser']==i.Loser))|((elo_hard['Loser']==i.Winner)&(elo_hard['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

In [328]:
for _,i in elo_data_clay.iterrows():
    check1=elo_clay[((elo_clay['Winner']==i.Winner)&(elo_clay['Loser']==i.Loser))|((elo_clay['Loser']==i.Winner)&(elo_clay['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Pablo Llamas Ruiz beat Alejandro Moro Canas
Pedro Cachin beat Andrea Collarini


In [329]:
for _,i in elo_data_grass.iterrows():
    check1=elo_grass[((elo_grass['Winner']==i.Winner)&(elo_grass['Loser']==i.Loser))|((elo_grass['Loser']==i.Winner)&(elo_grass['Winner']==i.Loser))]
    if check1.empty==False:
        for _, x in check1.iterrows():
            print(f"{x.Winner} beat {x.Loser}")

Viktorija Golubic beat Heather Watson


In [330]:
if final_hard is not None:
    final_hard=final_hard

else:
    final_hard=pd.DataFrame(columns=['Sex','Resulted','Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_hard[(final_hard['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,02:00,Gustavo Heide,Gustavo Heide,1.74,0.714286,423.0,0.0,0.0,0.8,Alex Rybakov,2.01,0.363636,424.0,0.0,0.0,0.4
1,03:30,Adam Walton,Adam Walton,1.29,0.600000,254.0,0.0,0.0,0.6,Rubin Statham,3.40,0.454545,464.0,0.0,0.0,0.4
2,06:30,Dane Sweeny,Bernard Tomic,1.74,0.500000,232.0,0.0,0.0,0.6,Bernard Tomic,2.00,0.400000,378.0,0.0,0.0,0.8


In [331]:
if final_grass is not None:
    final_grass=final_grass

else:
    final_grass=pd.DataFrame(columns=['Sex','Resulted','Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_grass[(final_grass['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,00:30,Dominic Stricker,Ryan Peniston,1.61,0.526316,117.0,0.86,0.13,0.50,Ryan Peniston,2.24,0.181818,201.0,0.80,0.19,0.4
1,00:30,Magda Linette,Magda Linette,1.46,0.529412,21.0,0.70,0.32,0.60,Anna Burrage Jodie,2.74,0.200000,131.0,NaN,NaN,0.0
2,01:00,Stefanos Tsitsipas,Stefanos Tsitsipas,1.29,0.692308,5.0,0.86,0.21,1.00,Richard Gasquet,3.62,0.500000,55.0,0.79,0.23,1.0
3,02:00,Andy Murray,Andy Murray,1.12,0.700000,44.0,0.79,0.24,1.00,Hugo Grenier,5.81,0.200000,127.0,0.77,0.17,0.4
4,19:00,Hubert Hurkacz,Hubert Hurkacz,1.38,0.735294,14.0,0.91,0.18,0.80,Christopher Oconnell,3.08,0.428571,74.0,0.79,0.16,0.6
5,19:00,Frances Tiafoe,Frances Tiafoe,1.46,0.448276,12.0,0.81,0.19,1.00,Lorenzo Musetti,2.72,0.384615,17.0,0.78,0.24,0.5
6,19:00,Taylor Fritz,Taylor Fritz,1.24,0.666667,8.0,0.84,0.24,1.00,Marton Fucsovics,4.04,0.400000,86.0,0.79,0.19,0.6
7,19:00,Liudmila Samsonova,Aliaksandra Sasnovich,1.42,0.684211,15.0,0.81,0.31,0.50,Aliaksandra Sasnovich,2.88,0.416667,71.0,0.67,0.38,0.5
8,19:00,Ekaterina Alexandrova,Ekaterina Alexandrova,1.20,0.642857,26.0,0.72,0.39,0.80,Emina Bektas,4.44,0.200000,186.0,0.58,0.10,0.8
9,19:00,Mackenzie Mcdonald,Rinky Hijikata,1.43,0.846154,68.0,0.77,0.24,0.40,Rinky Hijikata,2.85,0.333333,133.0,0.73,0.12,0.6


In [332]:
if final_clay is not None:
    final_clay=final_clay

else:
    final_clay=pd.DataFrame(columns=['Sex','Resulted','Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc'])

final_clay[(final_clay['Sex']=='Womens')&(final_clay['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
1,00:45,Sara Sorribes Tormo,Sara Sorribes Tormo,1.30,0.500000,82.0,0.59,0.46,0.8,Sara Errani,3.39,0.250000,69.0,0.5,0.4,0.6
3,03:15,Mayar Sherif,Mayar Sherif,1.17,0.272727,40.0,0.64,0.27,1.0,Lourdes Carle Maria,4.82,0.166667,140.0,NaN,NaN,0.4


In [333]:
final_clay[(final_clay['Sex']=='Mens')&(final_clay['Resulted']=='False')][['Time','Fav','Elo_Fav','Fav_Odds','fav_percent','fav_rank','Fav_Serve%','Fav_Return%','fav_last_five_win_perc','Dog','Dog_Odds','dog_percent','dog_rank','Dog_Serve%','Dog_Return%','dog_last_five_win_perc']]

,Time,Fav,Elo_Fav,Fav_Odds,fav_percent,fav_rank,Fav_Serve%,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,dog_rank,Dog_Serve%,Dog_Return%,dog_last_five_win_perc
0,00:30,Alex Molcan,Alex Molcan,1.21,0.461538,80.0,0.77,0.23,0.4,Kimmer Coppejans,4.18,0.300000,171.0,0.00,0.00,0.4
2,01:45,Benoit Paire,Lucas Pouille,1.86,0.818182,152.0,0.67,0.17,0.2,Lucas Pouille,1.87,0.500000,402.0,0.74,0.19,0.8
4,04:30,Pedro Cachin,Pedro Cachin,1.32,0.545455,71.0,0.78,0.19,0.4,Andrea Collarini,3.50,0.333333,220.0,0.00,0.00,0.4
5,18:00,Jaume Munar,Jaume Munar,1.47,0.473684,108.0,0.74,0.26,0.6,Alexandre Muller,2.57,0.461538,96.0,0.00,0.00,0.6
6,19:00,Felipe Meligeni Alves,Felipe Meligeni Alves,1.26,0.285714,156.0,0.14,0.25,0.6,Mathias Bourgue,3.70,0.400000,342.0,0.00,0.00,0.8
7,19:00,Alexander Ritschard,Alexander Ritschard,1.43,0.363636,195.0,0.76,0.17,0.6,Salvatore Caruso,2.75,0.500000,288.0,0.38,0.17,0.4
8,19:00,Jakub Mensik,Tomas Barrios Vera,1.72,0.500000,228.0,0.00,0.00,0.6,Tomas Barrios Vera,2.07,0.600000,161.0,0.62,0.00,0.6


In [334]:
if final_hard is not None:
    final_hard.to_pickle('Hard_Today')
else:
    final_clay[final_clay['Sex']=='k'].to_pickle('Hard_Today')
if final_grass is not None:
    final_grass.to_pickle('Grass_Today')
else:
    final_grass[final_grass['Sex']=='k'].to_pickle('Grass_Today')
if final_clay is not None:
    final_clay.to_pickle('Clay_Today')    
final_clay=final_clay[(final_clay['Resulted']=='False')].copy()
final_hard=final_hard[(final_hard['Resulted']=='False')].copy()    

model_file - Follow Prediction

In [335]:
data=final_clay
data['fav_win_percent_grt_10'] = data['fav_percent'] > 0.1
data['fav_win_percent_grt_20'] = data['fav_percent'] > 0.2
data['fav_win_percent_grt_30'] = data['fav_percent'] > 0.3
data['fav_win_percent_grt_40'] = data['fav_percent'] > 0.4
data['fav_win_percent_grt_50'] = data['fav_percent'] > 0.5
data['fav_win_percent_grt_60'] = data['fav_percent'] > 0.6
data['fav_win_percent_grt_70'] = data['fav_percent'] > 0.7
data['fav_win_percent_grt_80'] = data['fav_percent'] > 0.8
data['fav_win_percent_grt_90'] = data['fav_percent'] > 0.9
data['fav_win_percent_100'] = data['fav_percent'] == 1.0
data['dog_win_percent_grt_10'] = data['dog_percent'] > 0.1
data['dog_win_percent_grt_20'] = data['dog_percent'] > 0.2
data['dog_win_percent_grt_30'] = data['dog_percent'] > 0.3
data['dog_win_percent_grt_40'] = data['dog_percent'] > 0.4
data['dog_win_percent_grt_50'] = data['dog_percent'] > 0.5
data['dog_win_percent_grt_60'] = data['dog_percent'] > 0.6
data['dog_win_percent_grt_70'] = data['dog_percent'] > 0.7
data['dog_win_percent_grt_80'] = data['dog_percent'] > 0.8
data['dog_win_percent_grt_90'] = data['dog_percent'] > 0.9
data['dog_win_percent_100'] = data['dog_percent'] == 1.0

data['fav_win_percent_lt_10'] = data['fav_percent'] < 0.1
data['fav_win_percent_lt_20'] = data['fav_percent'] < 0.2
data['fav_win_percent_lt_30'] = data['fav_percent'] < 0.3
data['fav_win_percent_lt_40'] = data['fav_percent'] < 0.4
data['fav_win_percent_lt_50'] = data['fav_percent'] < 0.5
data['fav_win_percent_lt_60'] = data['fav_percent'] < 0.6
data['fav_win_percent_lt_70'] = data['fav_percent'] < 0.7
data['fav_win_percent_lt_80'] = data['fav_percent'] < 0.8
data['fav_win_percent_lt_90'] = data['fav_percent'] < 0.9
data['fav_win_percent_0'] = data['fav_percent'] == 0.0

data['dog_win_percent_lt_10'] = data['dog_percent'] < 0.1
data['dog_win_percent_lt_20'] = data['dog_percent'] < 0.2
data['dog_win_percent_lt_30'] = data['dog_percent'] < 0.3
data['dog_win_percent_lt_40'] = data['dog_percent'] < 0.4
data['dog_win_percent_lt_50'] = data['dog_percent'] < 0.5
data['dog_win_percent_lt_60'] = data['dog_percent'] < 0.6
data['dog_win_percent_lt_70'] = data['dog_percent'] < 0.7
data['dog_win_percent_lt_80'] = data['dog_percent'] < 0.8
data['dog_win_percent_lt_90'] = data['dog_percent'] < 0.9
data['dog_win_percent_0'] = data['dog_percent'] == 0.0
#data['Winner_']=data['Winner']
#data['Winner_IsFav']=data['Winner']==data['Fav']
#data['Fav_IsEloFav']=data['Fav']==data['Elo_Fav']
data['fav_rank_band'] = pd.cut(data['fav_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)
data['dog_rank_band'] = pd.cut(data['dog_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)

data['fav_rank_band'] = data['fav_rank_band'].astype(float)
data['dog_rank_band'] = data['dog_rank_band'].astype(float)

for i in range(0, 201, 50):
    data[f'fav_rank_gt_{i}'] = data['fav_rank_band'] > i
    data[f'fav_rank_lt_{i}'] = data['fav_rank_band'] < i

    data[f'dog_rank_gt_{i}'] = data['dog_rank_band'] > i
    data[f'dog_rank_lt_{i}'] = data['dog_rank_band'] < i





data['Female']=data['Sex']=='Womens'
data['Male']=data['Sex']=='Mens'
data=data[['Fav', 'fav_win_percent_grt_10',
       'fav_win_percent_grt_20', 'fav_win_percent_grt_30',
       'fav_win_percent_grt_40', 'fav_win_percent_grt_50',
       'fav_win_percent_grt_60', 'fav_win_percent_grt_70',
       'fav_win_percent_grt_80', 'fav_win_percent_grt_90',
       'fav_win_percent_100', 'dog_win_percent_grt_10',
       'dog_win_percent_grt_20', 'dog_win_percent_grt_30',
       'dog_win_percent_grt_40', 'dog_win_percent_grt_50',
       'dog_win_percent_grt_60', 'dog_win_percent_grt_70',
       'dog_win_percent_grt_80', 'dog_win_percent_grt_90',
       'dog_win_percent_100', 'fav_win_percent_lt_10', 'fav_win_percent_lt_20',
       'fav_win_percent_lt_30', 'fav_win_percent_lt_40',
       'fav_win_percent_lt_50', 'fav_win_percent_lt_60',
       'fav_win_percent_lt_70', 'fav_win_percent_lt_80',
       'fav_win_percent_lt_90', 'fav_win_percent_0', 'dog_win_percent_lt_10',
       'dog_win_percent_lt_20', 'dog_win_percent_lt_30',
       'dog_win_percent_lt_40', 'dog_win_percent_lt_50',
       'dog_win_percent_lt_60', 'dog_win_percent_lt_70',
       'dog_win_percent_lt_80', 'dog_win_percent_lt_90', 'dog_win_percent_0',
         'Female', 'Male','Fav_Odds','Time','fav_percent','dog_percent','fav_rank_gt_0',
 'fav_rank_lt_0',
 'dog_rank_gt_0',
 'dog_rank_lt_0',
 'fav_rank_gt_50',
 'fav_rank_lt_50',
 'dog_rank_gt_50',
 'dog_rank_lt_50',
 'fav_rank_gt_100',
 'fav_rank_lt_100',
 'dog_rank_gt_100',
 'dog_rank_lt_100',
 'fav_rank_gt_150',
 'fav_rank_lt_150',
 'dog_rank_gt_150',
 'dog_rank_lt_150',
 'fav_rank_gt_200',
 'fav_rank_lt_200',
 'dog_rank_gt_200',
 'dog_rank_lt_200',
         ]][(data['Fav_Odds']>1.8)]
filtered_data=data[['dog_win_percent_grt_20',
 'dog_rank_gt_100',
 'fav_win_percent_grt_30',
 'fav_win_percent_grt_40',
 'fav_win_percent_lt_40',
 'fav_rank_lt_200',
 'dog_win_percent_lt_70',
 'dog_rank_lt_100',
 'dog_win_percent_grt_50',
 'fav_rank_gt_50',
 'fav_win_percent_lt_30',
 'dog_rank_gt_0',
 'dog_rank_gt_150',
 'dog_rank_lt_200',
 'fav_rank_lt_50',
 'dog_win_percent_lt_80',
 'fav_rank_gt_100','Fav']][(data['Fav_Odds']>1.8)].copy()

from sklearn import svm
from joblib import load

if len(filtered_data)>0:
    # Load the saved model from the file
    model = load('.\model_file.joblib')

    # Assuming you have a new dataset stored in a variable called 'new_data'
    # Make predictions on the new dataset
    drop_fav=filtered_data.drop(columns=['Fav'])
    drop_fav=drop_fav.reset_index(drop=True)
    predictions = model.predict(drop_fav)
    predictions_df = pd.DataFrame(predictions, columns=['predictions'])

    # Merge the predictions with the original data based on the index
    #merged_data = pd.concat([drop_fav, predictions_df], axis=1)
    data_new_index=data.reset_index(drop=True)
    pred_final=predictions_df.merge(data_new_index,left_index=True,right_index=True)[['Fav','predictions','Fav_Odds','Time']]
else:
    pred_final=pd.DataFrame()

pred_final


,Fav,predictions,Fav_Odds,Time
0,Benoit Paire,True,1.86,01:45


In [336]:
data=final_clay
data['fav_win_percent_grt_10'] = data['fav_percent'] > 0.1
data['fav_win_percent_grt_20'] = data['fav_percent'] > 0.2
data['fav_win_percent_grt_30'] = data['fav_percent'] > 0.3
data['fav_win_percent_grt_40'] = data['fav_percent'] > 0.4
data['fav_win_percent_grt_50'] = data['fav_percent'] > 0.5
data['fav_win_percent_grt_60'] = data['fav_percent'] > 0.6
data['fav_win_percent_grt_70'] = data['fav_percent'] > 0.7
data['fav_win_percent_grt_80'] = data['fav_percent'] > 0.8
data['fav_win_percent_grt_90'] = data['fav_percent'] > 0.9
data['fav_win_percent_100'] = data['fav_percent'] == 1.0
data['dog_win_percent_grt_10'] = data['dog_percent'] > 0.1
data['dog_win_percent_grt_20'] = data['dog_percent'] > 0.2
data['dog_win_percent_grt_30'] = data['dog_percent'] > 0.3
data['dog_win_percent_grt_40'] = data['dog_percent'] > 0.4
data['dog_win_percent_grt_50'] = data['dog_percent'] > 0.5
data['dog_win_percent_grt_60'] = data['dog_percent'] > 0.6
data['dog_win_percent_grt_70'] = data['dog_percent'] > 0.7
data['dog_win_percent_grt_80'] = data['dog_percent'] > 0.8
data['dog_win_percent_grt_90'] = data['dog_percent'] > 0.9
data['dog_win_percent_100'] = data['dog_percent'] == 1.0

data['fav_win_percent_lt_10'] = data['fav_percent'] < 0.1
data['fav_win_percent_lt_20'] = data['fav_percent'] < 0.2
data['fav_win_percent_lt_30'] = data['fav_percent'] < 0.3
data['fav_win_percent_lt_40'] = data['fav_percent'] < 0.4
data['fav_win_percent_lt_50'] = data['fav_percent'] < 0.5
data['fav_win_percent_lt_60'] = data['fav_percent'] < 0.6
data['fav_win_percent_lt_70'] = data['fav_percent'] < 0.7
data['fav_win_percent_lt_80'] = data['fav_percent'] < 0.8
data['fav_win_percent_lt_90'] = data['fav_percent'] < 0.9
data['fav_win_percent_0'] = data['fav_percent'] == 0.0

data['dog_win_percent_lt_10'] = data['dog_percent'] < 0.1
data['dog_win_percent_lt_20'] = data['dog_percent'] < 0.2
data['dog_win_percent_lt_30'] = data['dog_percent'] < 0.3
data['dog_win_percent_lt_40'] = data['dog_percent'] < 0.4
data['dog_win_percent_lt_50'] = data['dog_percent'] < 0.5
data['dog_win_percent_lt_60'] = data['dog_percent'] < 0.6
data['dog_win_percent_lt_70'] = data['dog_percent'] < 0.7
data['dog_win_percent_lt_80'] = data['dog_percent'] < 0.8
data['dog_win_percent_lt_90'] = data['dog_percent'] < 0.9
data['dog_win_percent_0'] = data['dog_percent'] == 0.0
#data['Winner_']=data['Winner']
#data['Winner_IsFav']=data['Winner']==data['Fav']
#data['Fav_IsEloFav']=data['Fav']==data['Elo_Fav']
data['fav_rank_band'] = pd.cut(data['fav_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)
data['dog_rank_band'] = pd.cut(data['dog_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)

data['fav_rank_band'] = data['fav_rank_band'].astype(float)
data['dog_rank_band'] = data['dog_rank_band'].astype(float)

for i in range(0, 201, 50):
    data[f'fav_rank_gt_{i}'] = data['fav_rank_band'] > i
    data[f'fav_rank_lt_{i}'] = data['fav_rank_band'] < i

    data[f'dog_rank_gt_{i}'] = data['dog_rank_band'] > i
    data[f'dog_rank_lt_{i}'] = data['dog_rank_band'] < i





data['Female']=data['Sex']=='Womens'
data['Male']=data['Sex']=='Mens'
data=data[['Fav', 'fav_win_percent_grt_10',
       'fav_win_percent_grt_20', 'fav_win_percent_grt_30',
       'fav_win_percent_grt_40', 'fav_win_percent_grt_50',
       'fav_win_percent_grt_60', 'fav_win_percent_grt_70',
       'fav_win_percent_grt_80', 'fav_win_percent_grt_90',
       'fav_win_percent_100', 'dog_win_percent_grt_10',
       'dog_win_percent_grt_20', 'dog_win_percent_grt_30',
       'dog_win_percent_grt_40', 'dog_win_percent_grt_50',
       'dog_win_percent_grt_60', 'dog_win_percent_grt_70',
       'dog_win_percent_grt_80', 'dog_win_percent_grt_90',
       'dog_win_percent_100', 'fav_win_percent_lt_10', 'fav_win_percent_lt_20',
       'fav_win_percent_lt_30', 'fav_win_percent_lt_40',
       'fav_win_percent_lt_50', 'fav_win_percent_lt_60',
       'fav_win_percent_lt_70', 'fav_win_percent_lt_80',
       'fav_win_percent_lt_90', 'fav_win_percent_0', 'dog_win_percent_lt_10',
       'dog_win_percent_lt_20', 'dog_win_percent_lt_30',
       'dog_win_percent_lt_40', 'dog_win_percent_lt_50',
       'dog_win_percent_lt_60', 'dog_win_percent_lt_70',
       'dog_win_percent_lt_80', 'dog_win_percent_lt_90', 'dog_win_percent_0',
       'Female', 'Male','fav_percent','dog_percent','fav_rank_gt_0',
 'fav_rank_lt_0',
 'dog_rank_gt_0',
 'dog_rank_lt_0',
 'fav_rank_gt_50',
 'fav_rank_lt_50',
 'dog_rank_gt_50',
 'dog_rank_lt_50',
 'fav_rank_gt_100',
 'fav_rank_lt_100',
 'dog_rank_gt_100',
 'dog_rank_lt_100',
 'fav_rank_gt_150',
 'fav_rank_lt_150',
 'dog_rank_gt_150',
 'dog_rank_lt_150',
 'fav_rank_gt_200',
 'fav_rank_lt_200',
 'dog_rank_gt_200',
 'dog_rank_lt_200','Fav_Odds','Time'
         ]][(data['Fav_Odds']>1.7)&(data['Fav_Odds']<1.8)]

filtered_data=data[['fav_win_percent_grt_10', 'fav_win_percent_grt_20',
       'fav_win_percent_grt_30', 'fav_win_percent_grt_40',
       'fav_win_percent_grt_50', 'fav_win_percent_grt_60',
       'fav_win_percent_grt_70', 'fav_win_percent_grt_80',
       'fav_win_percent_grt_90', 'fav_win_percent_100',
       'dog_win_percent_grt_20', 'dog_win_percent_grt_30',
       'dog_win_percent_grt_40', 'dog_win_percent_grt_50',
       'dog_win_percent_grt_60', 'dog_win_percent_grt_70',
       'dog_win_percent_grt_90', 'dog_win_percent_100',
       'fav_win_percent_lt_10', 'fav_win_percent_lt_20',
       'fav_win_percent_lt_30', 'fav_win_percent_lt_40',
       'fav_win_percent_lt_50', 'fav_win_percent_lt_60',
       'fav_win_percent_lt_70', 'fav_win_percent_lt_80',
       'fav_win_percent_lt_90', 'fav_win_percent_0', 'dog_win_percent_lt_10',
       'dog_win_percent_lt_20', 'dog_win_percent_lt_30',
       'dog_win_percent_lt_40', 'dog_win_percent_lt_50',
       'dog_win_percent_lt_60', 'dog_win_percent_lt_70',
       'dog_win_percent_lt_80', 'dog_win_percent_lt_90', 'dog_win_percent_0',
         'Male', 'fav_rank_gt_0',
       'fav_rank_lt_0', 'dog_rank_gt_0', 'dog_rank_lt_0', 'fav_rank_gt_50',
       'fav_rank_lt_50', 'dog_rank_gt_50', 'dog_rank_lt_50', 'fav_rank_gt_100',
       'fav_rank_lt_100', 'dog_rank_gt_100', 'dog_rank_lt_100',
       'fav_rank_gt_150', 'fav_rank_lt_150', 'dog_rank_gt_150',
       'dog_rank_lt_150', 'fav_rank_gt_200', 'fav_rank_lt_200',
       'dog_rank_gt_200', 'dog_rank_lt_200', 'dog_percent','Fav',]].copy()

model_file2 - Follow Prediction

In [337]:
from sklearn import svm
from joblib import load
if len(filtered_data)>0:
    # Load the saved model from the file
    model = load('.\model_file2.joblib')

    # Assuming you have a new dataset stored in a variable called 'new_data'
    # Make predictions on the new dataset
    drop_fav=filtered_data.drop(columns=['Fav'])
    drop_fav=drop_fav.reset_index(drop=True)
    predictions = model.predict(drop_fav)
    predictions_df = pd.DataFrame(predictions, columns=['predictions'])

    # Merge the predictions with the original data based on the index
    #merged_data = pd.concat([drop_fav, predictions_df], axis=1)
    data_new_index=data.reset_index(drop=True)
    pred_final=predictions_df.merge(data_new_index,left_index=True,right_index=True)[['Fav','predictions','Fav_Odds','Time']]
else:
    pred_final=pd.DataFrame()

pred_final


,Fav,predictions,Fav_Odds,Time
0,Jakub Mensik,False,1.72,19:00


model_hard2 - Fade Prediction

In [338]:
data=final_hard
data['fav_win_percent_grt_10'] = data['fav_percent'] > 0.1
data['fav_win_percent_grt_20'] = data['fav_percent'] > 0.2
data['fav_win_percent_grt_30'] = data['fav_percent'] > 0.3
data['fav_win_percent_grt_40'] = data['fav_percent'] > 0.4
data['fav_win_percent_grt_50'] = data['fav_percent'] > 0.5
data['fav_win_percent_grt_60'] = data['fav_percent'] > 0.6
data['fav_win_percent_grt_70'] = data['fav_percent'] > 0.7
data['fav_win_percent_grt_80'] = data['fav_percent'] > 0.8
data['fav_win_percent_grt_90'] = data['fav_percent'] > 0.9
data['fav_win_percent_100'] = data['fav_percent'] == 1.0
data['dog_win_percent_grt_10'] = data['dog_percent'] > 0.1
data['dog_win_percent_grt_20'] = data['dog_percent'] > 0.2
data['dog_win_percent_grt_30'] = data['dog_percent'] > 0.3
data['dog_win_percent_grt_40'] = data['dog_percent'] > 0.4
data['dog_win_percent_grt_50'] = data['dog_percent'] > 0.5
data['dog_win_percent_grt_60'] = data['dog_percent'] > 0.6
data['dog_win_percent_grt_70'] = data['dog_percent'] > 0.7
data['dog_win_percent_grt_80'] = data['dog_percent'] > 0.8
data['dog_win_percent_grt_90'] = data['dog_percent'] > 0.9
data['dog_win_percent_100'] = data['dog_percent'] == 1.0

data['fav_win_percent_lt_10'] = data['fav_percent'] < 0.1
data['fav_win_percent_lt_20'] = data['fav_percent'] < 0.2
data['fav_win_percent_lt_30'] = data['fav_percent'] < 0.3
data['fav_win_percent_lt_40'] = data['fav_percent'] < 0.4
data['fav_win_percent_lt_50'] = data['fav_percent'] < 0.5
data['fav_win_percent_lt_60'] = data['fav_percent'] < 0.6
data['fav_win_percent_lt_70'] = data['fav_percent'] < 0.7
data['fav_win_percent_lt_80'] = data['fav_percent'] < 0.8
data['fav_win_percent_lt_90'] = data['fav_percent'] < 0.9
data['fav_win_percent_0'] = data['fav_percent'] == 0.0

data['dog_win_percent_lt_10'] = data['dog_percent'] < 0.1
data['dog_win_percent_lt_20'] = data['dog_percent'] < 0.2
data['dog_win_percent_lt_30'] = data['dog_percent'] < 0.3
data['dog_win_percent_lt_40'] = data['dog_percent'] < 0.4
data['dog_win_percent_lt_50'] = data['dog_percent'] < 0.5
data['dog_win_percent_lt_60'] = data['dog_percent'] < 0.6
data['dog_win_percent_lt_70'] = data['dog_percent'] < 0.7
data['dog_win_percent_lt_80'] = data['dog_percent'] < 0.8
data['dog_win_percent_lt_90'] = data['dog_percent'] < 0.9
data['dog_win_percent_0'] = data['dog_percent'] == 0.0
#data['Winner_']=data['Winner']
#data['Winner_IsFav']=data['Winner']==data['Fav']
#data['Fav_IsEloFav']=data['Fav']==data['Elo_Fav']
data['fav_rank_band'] = pd.cut(data['fav_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)
data['dog_rank_band'] = pd.cut(data['dog_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)

data['fav_rank_band'] = data['fav_rank_band'].astype(float)
data['dog_rank_band'] = data['dog_rank_band'].astype(float)

for i in range(0, 201, 50):
    data[f'fav_rank_gt_{i}'] = data['fav_rank_band'] > i
    data[f'fav_rank_lt_{i}'] = data['fav_rank_band'] < i

    data[f'dog_rank_gt_{i}'] = data['dog_rank_band'] > i
    data[f'dog_rank_lt_{i}'] = data['dog_rank_band'] < i





data['Female']=data['Sex']=='Womens'
data['Male']=data['Sex']=='Mens'
data=data[['Fav', 'fav_win_percent_grt_10',
       'fav_win_percent_grt_20', 'fav_win_percent_grt_30',
       'fav_win_percent_grt_40', 'fav_win_percent_grt_50',
       'fav_win_percent_grt_60', 'fav_win_percent_grt_70',
       'fav_win_percent_grt_80', 'fav_win_percent_grt_90',
       'fav_win_percent_100', 'dog_win_percent_grt_10',
       'dog_win_percent_grt_20', 'dog_win_percent_grt_30',
       'dog_win_percent_grt_40', 'dog_win_percent_grt_50',
       'dog_win_percent_grt_60', 'dog_win_percent_grt_70',
       'dog_win_percent_grt_80', 'dog_win_percent_grt_90',
       'dog_win_percent_100', 'fav_win_percent_lt_10', 'fav_win_percent_lt_20',
       'fav_win_percent_lt_30', 'fav_win_percent_lt_40',
       'fav_win_percent_lt_50', 'fav_win_percent_lt_60',
       'fav_win_percent_lt_70', 'fav_win_percent_lt_80',
       'fav_win_percent_lt_90', 'fav_win_percent_0', 'dog_win_percent_lt_10',
       'dog_win_percent_lt_20', 'dog_win_percent_lt_30',
       'dog_win_percent_lt_40', 'dog_win_percent_lt_50',
       'dog_win_percent_lt_60', 'dog_win_percent_lt_70',
       'dog_win_percent_lt_80', 'dog_win_percent_lt_90', 'dog_win_percent_0',
         'Female', 'Male','Fav_Odds','Time','fav_percent','dog_percent','fav_rank_gt_0',
 'fav_rank_lt_0',
 'dog_rank_gt_0',
 'dog_rank_lt_0',
 'fav_rank_gt_50',
 'fav_rank_lt_50',
 'dog_rank_gt_50',
 'dog_rank_lt_50',
 'fav_rank_gt_100',
 'fav_rank_lt_100',
 'dog_rank_gt_100',
 'dog_rank_lt_100',
 'fav_rank_gt_150',
 'fav_rank_lt_150',
 'dog_rank_gt_150',
 'dog_rank_lt_150',
 'fav_rank_gt_200',
 'fav_rank_lt_200',
 'dog_rank_gt_200',
 'dog_rank_lt_200',
         ]][(data['Fav_Odds']>1.8)]

from sklearn import svm
from joblib import load


if len(filtered_data)>0:
    # Load the saved model from the file
    model = load('.\model_hard2.joblib')

    # Assuming you have a new dataset stored in a variable called 'new_data'
    # Make predictions on the new dataset
    drop_fav=filtered_data.drop(columns=['Fav'])
    drop_fav=drop_fav.reset_index(drop=True)
    predictions = model.predict(drop_fav)
    predictions_df = pd.DataFrame(predictions, columns=['predictions'])

    # Merge the predictions with the original data based on the index
    #merged_data = pd.concat([drop_fav, predictions_df], axis=1)
    data_new_index=data.reset_index(drop=True)
    pred_final=predictions_df.merge(data_new_index,left_index=True,right_index=True)[['Fav','predictions','Fav_Odds','Time']]
else:
    pred_final=pd.DataFrame()

pred_final



,Fav,predictions,Fav_Odds,Time


model_hard - Fade Prediction

In [339]:
data=final_hard
data['fav_win_percent_grt_10'] = data['fav_percent'] > 0.1
data['fav_win_percent_grt_20'] = data['fav_percent'] > 0.2
data['fav_win_percent_grt_30'] = data['fav_percent'] > 0.3
data['fav_win_percent_grt_40'] = data['fav_percent'] > 0.4
data['fav_win_percent_grt_50'] = data['fav_percent'] > 0.5
data['fav_win_percent_grt_60'] = data['fav_percent'] > 0.6
data['fav_win_percent_grt_70'] = data['fav_percent'] > 0.7
data['fav_win_percent_grt_80'] = data['fav_percent'] > 0.8
data['fav_win_percent_grt_90'] = data['fav_percent'] > 0.9
data['fav_win_percent_100'] = data['fav_percent'] == 1.0
data['dog_win_percent_grt_10'] = data['dog_percent'] > 0.1
data['dog_win_percent_grt_20'] = data['dog_percent'] > 0.2
data['dog_win_percent_grt_30'] = data['dog_percent'] > 0.3
data['dog_win_percent_grt_40'] = data['dog_percent'] > 0.4
data['dog_win_percent_grt_50'] = data['dog_percent'] > 0.5
data['dog_win_percent_grt_60'] = data['dog_percent'] > 0.6
data['dog_win_percent_grt_70'] = data['dog_percent'] > 0.7
data['dog_win_percent_grt_80'] = data['dog_percent'] > 0.8
data['dog_win_percent_grt_90'] = data['dog_percent'] > 0.9
data['dog_win_percent_100'] = data['dog_percent'] == 1.0

data['fav_win_percent_lt_10'] = data['fav_percent'] < 0.1
data['fav_win_percent_lt_20'] = data['fav_percent'] < 0.2
data['fav_win_percent_lt_30'] = data['fav_percent'] < 0.3
data['fav_win_percent_lt_40'] = data['fav_percent'] < 0.4
data['fav_win_percent_lt_50'] = data['fav_percent'] < 0.5
data['fav_win_percent_lt_60'] = data['fav_percent'] < 0.6
data['fav_win_percent_lt_70'] = data['fav_percent'] < 0.7
data['fav_win_percent_lt_80'] = data['fav_percent'] < 0.8
data['fav_win_percent_lt_90'] = data['fav_percent'] < 0.9
data['fav_win_percent_0'] = data['fav_percent'] == 0.0

data['dog_win_percent_lt_10'] = data['dog_percent'] < 0.1
data['dog_win_percent_lt_20'] = data['dog_percent'] < 0.2
data['dog_win_percent_lt_30'] = data['dog_percent'] < 0.3
data['dog_win_percent_lt_40'] = data['dog_percent'] < 0.4
data['dog_win_percent_lt_50'] = data['dog_percent'] < 0.5
data['dog_win_percent_lt_60'] = data['dog_percent'] < 0.6
data['dog_win_percent_lt_70'] = data['dog_percent'] < 0.7
data['dog_win_percent_lt_80'] = data['dog_percent'] < 0.8
data['dog_win_percent_lt_90'] = data['dog_percent'] < 0.9
data['dog_win_percent_0'] = data['dog_percent'] == 0.0
#data['Winner_']=data['Winner']
#data['Winner_IsFav']=data['Winner']==data['Fav']
#data['Fav_IsEloFav']=data['Fav']==data['Elo_Fav']
data['fav_rank_band'] = pd.cut(data['fav_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)
data['dog_rank_band'] = pd.cut(data['dog_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)

data['fav_rank_band'] = data['fav_rank_band'].astype(float)
data['dog_rank_band'] = data['dog_rank_band'].astype(float)

for i in range(0, 201, 50):
    data[f'fav_rank_gt_{i}'] = data['fav_rank_band'] > i
    data[f'fav_rank_lt_{i}'] = data['fav_rank_band'] < i

    data[f'dog_rank_gt_{i}'] = data['dog_rank_band'] > i
    data[f'dog_rank_lt_{i}'] = data['dog_rank_band'] < i





data['Female']=data['Sex']=='Womens'
data['Male']=data['Sex']=='Mens'
data=data[['Fav', 'fav_win_percent_grt_10',
       'fav_win_percent_grt_20', 'fav_win_percent_grt_30',
       'fav_win_percent_grt_40', 'fav_win_percent_grt_50',
       'fav_win_percent_grt_60', 'fav_win_percent_grt_70',
       'fav_win_percent_grt_80', 'fav_win_percent_grt_90',
       'fav_win_percent_100', 'dog_win_percent_grt_10',
       'dog_win_percent_grt_20', 'dog_win_percent_grt_30',
       'dog_win_percent_grt_40', 'dog_win_percent_grt_50',
       'dog_win_percent_grt_60', 'dog_win_percent_grt_70',
       'dog_win_percent_grt_80', 'dog_win_percent_grt_90',
       'dog_win_percent_100', 'fav_win_percent_lt_10', 'fav_win_percent_lt_20',
       'fav_win_percent_lt_30', 'fav_win_percent_lt_40',
       'fav_win_percent_lt_50', 'fav_win_percent_lt_60',
       'fav_win_percent_lt_70', 'fav_win_percent_lt_80',
       'fav_win_percent_lt_90', 'fav_win_percent_0', 'dog_win_percent_lt_10',
       'dog_win_percent_lt_20', 'dog_win_percent_lt_30',
       'dog_win_percent_lt_40', 'dog_win_percent_lt_50',
       'dog_win_percent_lt_60', 'dog_win_percent_lt_70',
       'dog_win_percent_lt_80', 'dog_win_percent_lt_90', 'dog_win_percent_0',
         'Female', 'Male','Fav_Odds','Time','fav_percent','dog_percent','fav_rank_gt_0',
 'fav_rank_lt_0',
 'dog_rank_gt_0',
 'dog_rank_lt_0',
 'fav_rank_gt_50',
 'fav_rank_lt_50',
 'dog_rank_gt_50',
 'dog_rank_lt_50',
 'fav_rank_gt_100',
 'fav_rank_lt_100',
 'dog_rank_gt_100',
 'dog_rank_lt_100',
 'fav_rank_gt_150',
 'fav_rank_lt_150',
 'dog_rank_gt_150',
 'dog_rank_lt_150',
 'fav_rank_gt_200',
 'fav_rank_lt_200',
 'dog_rank_gt_200',
 'dog_rank_lt_200',
         ]][(data['Fav_Odds']>1.7)&(data['Fav_Odds']<1.8)]

from sklearn import svm
from joblib import load

if len(filtered_data)>0:
    # Load the saved model from the file
    model = load('.\model_hard.joblib')

    # Assuming you have a new dataset stored in a variable called 'new_data'
    # Make predictions on the new dataset
    drop_fav=filtered_data.drop(columns=['Fav'])
    drop_fav=drop_fav.reset_index(drop=True)
    predictions = model.predict(drop_fav)
    predictions_df = pd.DataFrame(predictions, columns=['predictions'])

    # Merge the predictions with the original data based on the index
    #merged_data = pd.concat([drop_fav, predictions_df], axis=1)
    data_new_index=data.reset_index(drop=True)
    pred_final=predictions_df.merge(data_new_index,left_index=True,right_index=True)[['Fav','predictions','Fav_Odds','Time']]
else:
    pred_final=pd.DataFrame()

pred_final


,Fav,predictions,Fav_Odds,Time
0,Gustavo Heide,True,1.74,02:00


model_clay14 - Follow Prediction True / Fade False

In [340]:

data=final_clay
data['fav_win_percent_grt_10'] = data['fav_percent'] > 0.1
data['fav_win_percent_grt_20'] = data['fav_percent'] > 0.2
data['fav_win_percent_grt_30'] = data['fav_percent'] > 0.3
data['fav_win_percent_grt_40'] = data['fav_percent'] > 0.4
data['fav_win_percent_grt_50'] = data['fav_percent'] > 0.5
data['fav_win_percent_grt_60'] = data['fav_percent'] > 0.6
data['fav_win_percent_grt_70'] = data['fav_percent'] > 0.7
data['fav_win_percent_grt_80'] = data['fav_percent'] > 0.8
data['fav_win_percent_grt_90'] = data['fav_percent'] > 0.9
data['fav_win_percent_100'] = data['fav_percent'] == 1.0
data['dog_win_percent_grt_10'] = data['dog_percent'] > 0.1
data['dog_win_percent_grt_20'] = data['dog_percent'] > 0.2
data['dog_win_percent_grt_30'] = data['dog_percent'] > 0.3
data['dog_win_percent_grt_40'] = data['dog_percent'] > 0.4
data['dog_win_percent_grt_50'] = data['dog_percent'] > 0.5
data['dog_win_percent_grt_60'] = data['dog_percent'] > 0.6
data['dog_win_percent_grt_70'] = data['dog_percent'] > 0.7
data['dog_win_percent_grt_80'] = data['dog_percent'] > 0.8
data['dog_win_percent_grt_90'] = data['dog_percent'] > 0.9
data['dog_win_percent_100'] = data['dog_percent'] == 1.0

data['fav_win_percent_lt_10'] = data['fav_percent'] < 0.1
data['fav_win_percent_lt_20'] = data['fav_percent'] < 0.2
data['fav_win_percent_lt_30'] = data['fav_percent'] < 0.3
data['fav_win_percent_lt_40'] = data['fav_percent'] < 0.4
data['fav_win_percent_lt_50'] = data['fav_percent'] < 0.5
data['fav_win_percent_lt_60'] = data['fav_percent'] < 0.6
data['fav_win_percent_lt_70'] = data['fav_percent'] < 0.7
data['fav_win_percent_lt_80'] = data['fav_percent'] < 0.8
data['fav_win_percent_lt_90'] = data['fav_percent'] < 0.9
data['fav_win_percent_0'] = data['fav_percent'] == 0.0

data['dog_win_percent_lt_10'] = data['dog_percent'] < 0.1
data['dog_win_percent_lt_20'] = data['dog_percent'] < 0.2
data['dog_win_percent_lt_30'] = data['dog_percent'] < 0.3
data['dog_win_percent_lt_40'] = data['dog_percent'] < 0.4
data['dog_win_percent_lt_50'] = data['dog_percent'] < 0.5
data['dog_win_percent_lt_60'] = data['dog_percent'] < 0.6
data['dog_win_percent_lt_70'] = data['dog_percent'] < 0.7
data['dog_win_percent_lt_80'] = data['dog_percent'] < 0.8
data['dog_win_percent_lt_90'] = data['dog_percent'] < 0.9
data['dog_win_percent_0'] = data['dog_percent'] == 0.0
#data['Winner_']=data['Winner']
#data['Winner_IsFav']=data['Winner']==data['Fav']
#data['Fav_IsEloFav']=data['Fav']==data['Elo_Fav']
data['fav_rank_band'] = pd.cut(data['fav_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)
data['dog_rank_band'] = pd.cut(data['dog_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)

data['fav_rank_band'] = data['fav_rank_band'].astype(float)
data['dog_rank_band'] = data['dog_rank_band'].astype(float)

for i in range(0, 201, 50):
    data[f'fav_rank_gt_{i}'] = data['fav_rank_band'] > i
    data[f'fav_rank_lt_{i}'] = data['fav_rank_band'] < i

    data[f'dog_rank_gt_{i}'] = data['dog_rank_band'] > i
    data[f'dog_rank_lt_{i}'] = data['dog_rank_band'] < i





data['Female']=data['Sex']=='Womens'
data['Male']=data['Sex']=='Mens'
data=data[['Fav', 'fav_win_percent_grt_10',
       'fav_win_percent_grt_20', 'fav_win_percent_grt_30',
       'fav_win_percent_grt_40', 'fav_win_percent_grt_50',
       'fav_win_percent_grt_60', 'fav_win_percent_grt_70',
       'fav_win_percent_grt_80', 'fav_win_percent_grt_90',
       'fav_win_percent_100', 'dog_win_percent_grt_10',
       'dog_win_percent_grt_20', 'dog_win_percent_grt_30',
       'dog_win_percent_grt_40', 'dog_win_percent_grt_50',
       'dog_win_percent_grt_60', 'dog_win_percent_grt_70',
       'dog_win_percent_grt_80', 'dog_win_percent_grt_90',
       'dog_win_percent_100', 'fav_win_percent_lt_10', 'fav_win_percent_lt_20',
       'fav_win_percent_lt_30', 'fav_win_percent_lt_40',
       'fav_win_percent_lt_50', 'fav_win_percent_lt_60',
       'fav_win_percent_lt_70', 'fav_win_percent_lt_80',
       'fav_win_percent_lt_90', 'fav_win_percent_0', 'dog_win_percent_lt_10',
       'dog_win_percent_lt_20', 'dog_win_percent_lt_30',
       'dog_win_percent_lt_40', 'dog_win_percent_lt_50',
       'dog_win_percent_lt_60', 'dog_win_percent_lt_70',
       'dog_win_percent_lt_80', 'dog_win_percent_lt_90', 'dog_win_percent_0',
         'Female', 'Male','Fav_Odds','Time','fav_percent','dog_percent','fav_rank_gt_0',
 'fav_rank_lt_0',
 'dog_rank_gt_0',
 'dog_rank_lt_0',
 'fav_rank_gt_50',
 'fav_rank_lt_50',
 'dog_rank_gt_50',
 'dog_rank_lt_50',
 'fav_rank_gt_100',
 'fav_rank_lt_100',
 'dog_rank_gt_100',
 'dog_rank_lt_100',
 'fav_rank_gt_150',
 'fav_rank_lt_150',
 'dog_rank_gt_150',
 'dog_rank_lt_150',
 'fav_rank_gt_200',
 'fav_rank_lt_200',
 'dog_rank_gt_200',
 'dog_rank_lt_200',
         ]][(data['Fav_Odds']>1.4)&(data['Fav_Odds']<1.5)]

from sklearn import svm
from joblib import load


if len(filtered_data)>0:
    # Load the saved model from the file
    model = load('.\model_clay14.joblib')

    # Assuming you have a new dataset stored in a variable called 'new_data'
    # Make predictions on the new dataset
    drop_fav=filtered_data.drop(columns=['Fav'])
    drop_fav=drop_fav.reset_index(drop=True)
    predictions = model.predict(drop_fav)
    predictions_df = pd.DataFrame(predictions, columns=['predictions'])

    # Merge the predictions with the original data based on the index
    #merged_data = pd.concat([drop_fav, predictions_df], axis=1)
    data_new_index=data.reset_index(drop=True)
    pred_final=predictions_df.merge(data_new_index,left_index=True,right_index=True)[['Fav','predictions','Fav_Odds','Time']]
else:
    pred_final=pd.DataFrame()

pred_final


,Fav,predictions,Fav_Odds,Time
0,Jaume Munar,True,1.47,18:00


In [341]:

data=final_clay
data['fav_win_percent_grt_10'] = data['fav_percent'] > 0.1
data['fav_win_percent_grt_20'] = data['fav_percent'] > 0.2
data['fav_win_percent_grt_30'] = data['fav_percent'] > 0.3
data['fav_win_percent_grt_40'] = data['fav_percent'] > 0.4
data['fav_win_percent_grt_50'] = data['fav_percent'] > 0.5
data['fav_win_percent_grt_60'] = data['fav_percent'] > 0.6
data['fav_win_percent_grt_70'] = data['fav_percent'] > 0.7
data['fav_win_percent_grt_80'] = data['fav_percent'] > 0.8
data['fav_win_percent_grt_90'] = data['fav_percent'] > 0.9
data['fav_win_percent_100'] = data['fav_percent'] == 1.0
data['dog_win_percent_grt_10'] = data['dog_percent'] > 0.1
data['dog_win_percent_grt_20'] = data['dog_percent'] > 0.2
data['dog_win_percent_grt_30'] = data['dog_percent'] > 0.3
data['dog_win_percent_grt_40'] = data['dog_percent'] > 0.4
data['dog_win_percent_grt_50'] = data['dog_percent'] > 0.5
data['dog_win_percent_grt_60'] = data['dog_percent'] > 0.6
data['dog_win_percent_grt_70'] = data['dog_percent'] > 0.7
data['dog_win_percent_grt_80'] = data['dog_percent'] > 0.8
data['dog_win_percent_grt_90'] = data['dog_percent'] > 0.9
data['dog_win_percent_100'] = data['dog_percent'] == 1.0

data['fav_win_percent_lt_10'] = data['fav_percent'] < 0.1
data['fav_win_percent_lt_20'] = data['fav_percent'] < 0.2
data['fav_win_percent_lt_30'] = data['fav_percent'] < 0.3
data['fav_win_percent_lt_40'] = data['fav_percent'] < 0.4
data['fav_win_percent_lt_50'] = data['fav_percent'] < 0.5
data['fav_win_percent_lt_60'] = data['fav_percent'] < 0.6
data['fav_win_percent_lt_70'] = data['fav_percent'] < 0.7
data['fav_win_percent_lt_80'] = data['fav_percent'] < 0.8
data['fav_win_percent_lt_90'] = data['fav_percent'] < 0.9
data['fav_win_percent_0'] = data['fav_percent'] == 0.0

data['dog_win_percent_lt_10'] = data['dog_percent'] < 0.1
data['dog_win_percent_lt_20'] = data['dog_percent'] < 0.2
data['dog_win_percent_lt_30'] = data['dog_percent'] < 0.3
data['dog_win_percent_lt_40'] = data['dog_percent'] < 0.4
data['dog_win_percent_lt_50'] = data['dog_percent'] < 0.5
data['dog_win_percent_lt_60'] = data['dog_percent'] < 0.6
data['dog_win_percent_lt_70'] = data['dog_percent'] < 0.7
data['dog_win_percent_lt_80'] = data['dog_percent'] < 0.8
data['dog_win_percent_lt_90'] = data['dog_percent'] < 0.9
data['dog_win_percent_0'] = data['dog_percent'] == 0.0
#data['Winner_']=data['Winner']
#data['Winner_IsFav']=data['Winner']==data['Fav']
#data['Fav_IsEloFav']=data['Fav']==data['Elo_Fav']
data['fav_rank_band'] = pd.cut(data['fav_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)
data['dog_rank_band'] = pd.cut(data['dog_rank'], bins=list(range(0, 201, 20)), labels=list(range(20, 201, 20)), right=False)

data['fav_rank_band'] = data['fav_rank_band'].astype(float)
data['dog_rank_band'] = data['dog_rank_band'].astype(float)

for i in range(0, 201, 50):
    data[f'fav_rank_gt_{i}'] = data['fav_rank_band'] > i
    data[f'fav_rank_lt_{i}'] = data['fav_rank_band'] < i

    data[f'dog_rank_gt_{i}'] = data['dog_rank_band'] > i
    data[f'dog_rank_lt_{i}'] = data['dog_rank_band'] < i





data['Female']=data['Sex']=='Womens'
data['Male']=data['Sex']=='Mens'
data=data[['Fav', 'fav_win_percent_grt_10',
       'fav_win_percent_grt_20', 'fav_win_percent_grt_30',
       'fav_win_percent_grt_40', 'fav_win_percent_grt_50',
       'fav_win_percent_grt_60', 'fav_win_percent_grt_70',
       'fav_win_percent_grt_80', 'fav_win_percent_grt_90',
       'fav_win_percent_100', 'dog_win_percent_grt_10',
       'dog_win_percent_grt_20', 'dog_win_percent_grt_30',
       'dog_win_percent_grt_40', 'dog_win_percent_grt_50',
       'dog_win_percent_grt_60', 'dog_win_percent_grt_70',
       'dog_win_percent_grt_80', 'dog_win_percent_grt_90',
       'dog_win_percent_100', 'fav_win_percent_lt_10', 'fav_win_percent_lt_20',
       'fav_win_percent_lt_30', 'fav_win_percent_lt_40',
       'fav_win_percent_lt_50', 'fav_win_percent_lt_60',
       'fav_win_percent_lt_70', 'fav_win_percent_lt_80',
       'fav_win_percent_lt_90', 'fav_win_percent_0', 'dog_win_percent_lt_10',
       'dog_win_percent_lt_20', 'dog_win_percent_lt_30',
       'dog_win_percent_lt_40', 'dog_win_percent_lt_50',
       'dog_win_percent_lt_60', 'dog_win_percent_lt_70',
       'dog_win_percent_lt_80', 'dog_win_percent_lt_90', 'dog_win_percent_0',
         'Female', 'Male','Fav_Odds','Time','fav_percent','dog_percent','fav_rank_gt_0',
 'fav_rank_lt_0',
 'dog_rank_gt_0',
 'dog_rank_lt_0',
 'fav_rank_gt_50',
 'fav_rank_lt_50',
 'dog_rank_gt_50',
 'dog_rank_lt_50',
 'fav_rank_gt_100',
 'fav_rank_lt_100',
 'dog_rank_gt_100',
 'dog_rank_lt_100',
 'fav_rank_gt_150',
 'fav_rank_lt_150',
 'dog_rank_gt_150',
 'dog_rank_lt_150',
 'fav_rank_gt_200',
 'fav_rank_lt_200',
 'dog_rank_gt_200',
 'dog_rank_lt_200',
         ]][(data['Fav_Odds']>=1.5)&(data['Fav_Odds']<=1.6)]

from sklearn import svm
from joblib import load


if len(filtered_data)>0:
    # Load the saved model from the file
    model = load('.\model_clay15.joblib')

    # Assuming you have a new dataset stored in a variable called 'new_data'
    # Make predictions on the new dataset
    drop_fav=filtered_data.drop(columns=['Fav'])
    drop_fav=drop_fav.reset_index(drop=True)
    predictions = model.predict(drop_fav)
    predictions_df = pd.DataFrame(predictions, columns=['predictions'])

    # Merge the predictions with the original data based on the index
    #merged_data = pd.concat([drop_fav, predictions_df], axis=1)
    data_new_index=data.reset_index(drop=True)
    pred_final=predictions_df.merge(data_new_index,left_index=True,right_index=True)[['Fav','predictions','Fav_Odds','Time']]
else:
    pred_final=pd.DataFrame()

pred_final


,Fav,predictions,Fav_Odds,Time
